In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./part.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Vehicle_Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_tr = html.xpath('//tr[@class="sortable_row filter-field"]')
                
                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                for tr in list_tr:
                    make = tr.xpath('./td[2]/text()')[0].strip()
                    model = tr.xpath('./td[3]/text()')[0].strip()
                    year_begin = int(tr.xpath('./td[1]/text()')[0].split('-')[0].strip())
                    year_end = int(tr.xpath('./td[1]/text()')[0].split('-')[-1].strip())

                    if make not in dict_vehicle:
                        dict_vehicle[make] = {model: [year_begin, year_end]}
                    else:
                        if model not in dict_vehicle[make]:
                            dict_vehicle[make][model] = [year_begin, year_end]
                        else:
                            dict_vehicle[make][model].append(year_begin)
                            dict_vehicle[make][model].append(year_end)
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        year_max = max(dict_vehicle[make][model])
                        year_min = min(dict_vehicle[make][model])
                        if year_max == year_min:
                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                        else:
                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//table[@id="interchange-codes-table"]/tbody/tr')
                
                # = = = = = = = = = = = = = = =
                
                list_oe = []
                for tr in list_tr:
                    list_oe.append(tr.xpath('./td[2]/text()')[0].strip())

                list_oe = list(set(list_oe))
                list_oe.sort()

                oe = ';'.join(list_oe)

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Vehicle_Url': url,
                                         'Vehicle': vehicle,
                                         'OE': oe}])
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Vehicle_Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./vehicle_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：6586

https://www.1aauto.com/catalog/product/fit?skuId=1218667&pathName=%2F2000-subaru-outback-from-8-01-00-production-date-front-driver-and-passenger-side-semi-metallic-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-bka29858%2Fi%2F1abfs16430  <->  [ok] - 剩余数量：6576 - 当前时间：13:45:18
https://www.1aauto.com/catalog/product/fit?skuId=1179366&pathName=%2F1990-02-chevrolet-astro-gmc-safari-front-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka18389%2Fi%2F1abfs08752  <->  [ok] - 剩余数量：6575 - 当前时间：13:45:18
https://www.1aauto.com/catalog/product/fit?skuId=1184088&pathName=%2F1998-02-ford-lincoln-mercury-front-and-rear-semi-metallic-brake-pads-trq-bfa20292%2Fi%2F1abfs10057  <->  [ok] - 剩余数量：6574 - 当前时间：13:45:18
https://www.1aauto.com/catalog/product/fit?skuId=1218703&pathName=%2F1996-00-chevrolet-k3500-gmc-k3500-front-semi-metallic-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-bka29768%2Fi%2F1abfs16341  <->  [ok] - 剩余数量：6573 - 当前时间：13:45:18
https://www.1aauto.com/catalog/produc

https://www.1aauto.com/catalog/product/fit?skuId=1191293&pathName=%2F2004-ford-f150-truck-with-new-body-style-rear-wheel-drive-to-10-31-04-production-date-with-6-lug-wheels-with-caliper-casting-%23262266-front-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka25049%2Fi%2F1abfs13033  <->  [ok] - 剩余数量：6542 - 当前时间：13:45:24
https://www.1aauto.com/catalog/product/fit?skuId=1191324&pathName=%2F2004-ford-f150-truck-with-new-body-style-rear-wheel-drive-to-10-31-04-production-date-with-6-lug-wheels-with-caliper-casting-%23262266-front-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka25510%2Fi%2F1abfs13480  <->  [ok] - 剩余数量：6541 - 当前时间：13:45:24
https://www.1aauto.com/catalog/product/fit?skuId=1211784&pathName=%2F1999-04-jeep-grand-cherokee-rear-4-piece-brake-caliper-set-with-hoses-trq-cla41143%2Fi%2F1abcs00701  <->  [ok] - 剩余数量：6540 - 当前时间：13:45:24
https://www.1aauto.com/catalog/product/fit?skuId=1186812&pathName=%2F2002-04-honda-cr-v-rear-ceramic-performance-brake-pad-and-rotor-kit-with-

https://www.1aauto.com/catalog/product/fit?skuId=1191081&pathName=%2F2002-hyundai-elantra-to-10-31-01-production-date-front-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka25250%2Fi%2F1abfs13228  <->  [ok] - 剩余数量：6509 - 当前时间：13:45:30
https://www.1aauto.com/catalog/product/fit?skuId=1169610&pathName=%2F1998-02-chevrolet-prizm-toyota-corolla-front-driver-and-passenger-side-2-piece-brake-caliper-set-trq-cla35426%2Fi%2F1abcs00093  <->  [ok] - 剩余数量：6508 - 当前时间：13:45:31
https://www.1aauto.com/catalog/product/fit?skuId=1216183&pathName=%2F1998-02-chevrolet-prizm-toyota-corolla-front-driver-and-passenger-side-4-piece-brake-caliper-set-with-hoses-trq-cla41333%2Fi%2F1abcs00847  <->  [ok] - 剩余数量：6507 - 当前时间：13:45:31
https://www.1aauto.com/catalog/product/fit?skuId=1169649&pathName=%2F1999-02-jeep-grand-cherokee-with-teves-calipers-front-driver-and-passenger-side-2-piece-brake-caliper-set-trq-cla35417%2Fi%2F1abcs00108  <->  [ok] - 剩余数量：6506 - 当前时间：13:45:31
https://www.1aauto.com/catalog/produ

https://www.1aauto.com/catalog/product/fit?skuId=1129907&pathName=%2F1998-02-honda-accord-coupe-l4-2.3l-rear-ceramic-performance-brake-pad-and-rotor-kit-with-calipers-diy-solutions-bfs01986%2Fi%2F1apbs01091  <->  [ok] - 剩余数量：6473 - 当前时间：13:45:38
https://www.1aauto.com/catalog/product/fit?skuId=1129918&pathName=%2F1998-02-honda-accord-coupe-l4-2.3l-rear-ceramic-brake-pad-and-rotor-kit-with-calipers-diy-solutions-bfs00983%2Fi%2F1abfs03072  <->  [ok] - 剩余数量：6472 - 当前时间：13:45:39
https://www.1aauto.com/catalog/product/fit?skuId=1221731&pathName=%2F1999-02-ford-mustang-front-ceramic-performance-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-performance-bka34137%2Fi%2F1apbs07359  <->  [ok] - 剩余数量：6471 - 当前时间：13:45:39
https://www.1aauto.com/catalog/product/fit?skuId=1179959&pathName=%2F1998-02-chevrolet-prizm-toyota-corolla-front-semi-metallic-performance-brake-pad-and-rotor-kit-with-calipers-trq-performance-bka18429%2Fi%2F1apbs02504  <->  [ok] - 剩余数量：6470 - 当前时间：13:45:39
https://www.1aau

https://www.1aauto.com/catalog/product/fit?skuId=1194302&pathName=%2F1996-00-dodge-chrysler-plymouth-front-driver-and-passenger-side-semi-metallic-performance-brake-pad-and-rotor-kit-trq-performance-bka21617%2Fi%2F1apbs04135  <->  [ok] - 剩余数量：6440 - 当前时间：13:45:53
https://www.1aauto.com/catalog/product/fit?skuId=1220425&pathName=%2F2004-ford-f150-truck-with-new-body-style-rear-wheel-drive-to-10-31-04-production-date-with-6-lug-wheels-with-caliper-casting-%23262266-front-driver-and-passenger-side-ceramic-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-bka29929%2Fi%2F1abfs16501  <->  [ok] - 剩余数量：6439 - 当前时间：13:45:54
https://www.1aauto.com/catalog/product/fit?skuId=1218653&pathName=%2F2004-ford-f150-truck-with-new-body-style-rear-wheel-drive-to-10-31-04-production-date-with-6-lug-wheels-with-caliper-casting-%23262266-front-driver-and-passenger-side-semi-metallic-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-bka29928%2Fi%2F1abfs16500  <->  [ok] - 剩余数量：6438 - 当前时间：13:45:54
https://

...

https://www.1aauto.com/catalog/product/fit?skuId=1192248&pathName=%2F2007-chevrolet-gmc-rear-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka26378%2Fi%2F1abfs13701  <->  [ok] - 剩余数量：2695 - 当前时间：14:14:26
https://www.1aauto.com/catalog/product/fit?skuId=1192244&pathName=%2Fvw-cc-passat-rear-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka26380%2Fi%2F1abfs13703  <->  [ok] - 剩余数量：2694 - 当前时间：14:14:26
https://www.1aauto.com/catalog/product/fit?skuId=1192247&pathName=%2Fhonda-accord-rear-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka26381%2Fi%2F1abfs13704  <->  [ok] - 剩余数量：2693 - 当前时间：14:14:27
https://www.1aauto.com/catalog/product/fit?skuId=1190893&pathName=%2Fford-escape-mercury-mariner-front-semi-metallic-brake-pad-and-rotor-kit-with-calipers-trq-bka25656%2Fi%2F1abfs13627  <->  [ok] - 剩余数量：2692 - 当前时间：14:14:27
https://www.1aauto.com/catalog/product/fit?skuId=1192240&pathName=%2Fmazda-5-rear-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka26384%2Fi%2F1abfs13707  <->  

https://www.1aauto.com/catalog/product/fit?skuId=1192194&pathName=%2Fdodge-ram-3500-rear-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka26455%2Fi%2F1abfs13777  <->  [ok] - 剩余数量：2653 - 当前时间：14:14:40
https://www.1aauto.com/catalog/product/fit?skuId=1192243&pathName=%2Fhyundai-sonata-kia-optima-rear-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka26385%2Fi%2F1abfs13708  <->  [ok] - 剩余数量：2652 - 当前时间：14:14:40
https://www.1aauto.com/catalog/product/fit?skuId=1192278&pathName=%2F2014-toyota-highlander-hybrid-limited-rear-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka26470%2Fi%2F1abfs13792  <->  [ok] - 剩余数量：2651 - 当前时间：14:14:40
https://www.1aauto.com/catalog/product/fit?skuId=1191004&pathName=%2Fsubaru-impreza-legacy-saab-9-2x-front-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka25610%2Fi%2F1abfs13581  <->  [ok] - 剩余数量：2650 - 当前时间：14:14:41
https://www.1aauto.com/catalog/product/fit?skuId=1192260&pathName=%2Fford-f150-rear-ceramic-brake-pad-and-rotor-kit-with-calipers-tr

https://www.1aauto.com/catalog/product/fit?skuId=1191036&pathName=%2Fdodge-caliber-front-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka25601%2Fi%2F1abfs13572  <->  [ok] - 剩余数量：2614 - 当前时间：14:14:50
https://www.1aauto.com/catalog/product/fit?skuId=1190915&pathName=%2Fhonda-civic-front-semi-metallic-brake-pad-and-rotor-kit-with-calipers-trq-bka25634%2Fi%2F1abfs13605  <->  [ok] - 剩余数量：2613 - 当前时间：14:14:50
https://www.1aauto.com/catalog/product/fit?skuId=1190912&pathName=%2Ftoyota-pontiac-scion-front-semi-metallic-brake-pad-and-rotor-kit-with-calipers-trq-bka25633%2Fi%2F1abfs13604  <->  [ok] - 剩余数量：2612 - 当前时间：14:14:50
https://www.1aauto.com/catalog/product/fit?skuId=1190908&pathName=%2Ftoyota-rav4-front-semi-metallic-brake-pad-and-rotor-kit-with-calipers-trq-bka25751%2Fi%2F1abfs13630  <->  [ok] - 剩余数量：2611 - 当前时间：14:14:50
https://www.1aauto.com/catalog/product/fit?skuId=1190885&pathName=%2Fford-edge-lincoln-mkx-front-semi-metallic-brake-pad-and-rotor-kit-with-calipers-trq-bka25772%2

https://www.1aauto.com/catalog/product/fit?skuId=1192142&pathName=%2Fkia-forte-forte-koup-rear-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka26507%2Fi%2F1abfs13829  <->  [ok] - 剩余数量：2575 - 当前时间：14:15:04
https://www.1aauto.com/catalog/product/fit?skuId=1192111&pathName=%2Fchevrolet-camaro-rear-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka26555%2Fi%2F1abfs13878  <->  [ok] - 剩余数量：2574 - 当前时间：14:15:04
https://www.1aauto.com/catalog/product/fit?skuId=1192156&pathName=%2Fpontiac-grand-am-olds-alero-rear-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka26509%2Fi%2F1abfs13831  <->  [ok] - 剩余数量：2573 - 当前时间：14:15:04
https://www.1aauto.com/catalog/product/fit?skuId=1192157&pathName=%2Fjeep-grand-cherokee-rear-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka26508%2Fi%2F1abfs13830  <->  [ok] - 剩余数量：2572 - 当前时间：14:15:05
https://www.1aauto.com/catalog/product/fit?skuId=1190900&pathName=%2Fvw-audi-front-semi-metallic-brake-pad-and-rotor-kit-with-calipers-trq-bka25757%2Fi%2F1abf

https://www.1aauto.com/catalog/product/fit?skuId=1191969&pathName=%2Fnissan-infiniti-rear-semi-metallic-brake-pad-and-rotor-kit-with-calipers-trq-bka26669%2Fi%2F1abfs13989  <->  [ok] - 剩余数量：2536 - 当前时间：14:15:20
